# Demos: Lecture 19

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

## Demo 1: Pauli circuit decomposition

In [ ]:
dev = qml.device('default.qubit', wires=4)

@qml.compile(pipeline=[], basis_set=["RX", "RY", "RZ", "Hadamard", "CNOT"])
@qml.qnode(dev)
def pauli_rot(t):
    for wire in dev.wires:
        qml.Hadamard(wires=wire)
    qml.evolve(qml.PauliX(0) @ qml.PauliY(1) @ qml.PauliZ(3), t)
    qml.evolve(qml.PauliX(0) @ qml.PauliY(2) @ qml.PauliZ(3), t)
    return qml.expval(qml.PauliX(wires=0) @ qml.PauliX(wires=1))

In [ ]:
print(qml.draw(pauli_rot)(0.5))

In [ ]:
times = np.linspace(0, 10, 100)
results = [pauli_rot(t) for t in times]
plt.plot(times, results)

## Demo 2: commutation chaos

In [ ]:
H = qml.Hamiltonian([1, 1], [qml.PauliZ(0), qml.PauliX(0)])

def construct_unitary(t):
    return expm(-1j * t * qml.matrix(H))

In [ ]:
dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def change_pauli_order(t, exact=False, reverse=False):
    qml.Hadamard(wires=0)

    if exact:
        qml.QubitUnitary(construct_unitary(t), wires=0)
    else:
        if reverse:
            qml.evolve(qml.PauliX(0), t)
            qml.evolve(qml.PauliZ(0), t)
        else:
            qml.evolve(qml.PauliZ(0), t)
            qml.evolve(qml.PauliX(0), t)
    
    return qml.expval(qml.PauliY(wires=0))

In [ ]:
results_exact = [change_pauli_order(t, exact=True) for t in times]
results = [change_pauli_order(t) for t in times]
results_reversed = [change_pauli_order(t, reverse=True) for t in times]

plt.figure(figsize=(12, 8))
plt.plot(times, results_exact, label="True")
plt.plot(times, results, label="Original")
plt.plot(times, results_reversed, label="Reversed")
plt.legend(fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Evolution time", fontsize=14)
plt.ylabel("<Y>", fontsize=14)
plt.show()

## Demo 3: Trotterization

In [ ]:
dev = qml.device('default.qubit', wires=1)

@qml.compile(pipeline=[], basis_set=["RX", "RY", "RZ", "Hadamard", "CNOT"])
@qml.qnode(dev)
def trotterize(t, exact=False, trotter_steps=1):
    qml.Hadamard(wires=0)

    if exact:
        qml.QubitUnitary(construct_unitary(t), wires=0)
    else:
        qml.exp(H, -1j * t, num_steps=trotter_steps)
    
    return qml.expval(qml.PauliY(wires=0))

In [ ]:
print(qml.draw(trotterize)(0.5, exact=False, trotter_steps=5))

In [ ]:
results_exact = [trotterize(t, exact=True) for t in times]
results_1_step = [trotterize(t, trotter_steps=1) for t in times]
results_5_step = [trotterize(t, trotter_steps=5) for t in times]
results_10_step = [trotterize(t, trotter_steps=10) for t in times]
results_20_step = [trotterize(t, trotter_steps=20) for t in times]
results_100_step = [trotterize(t, trotter_steps=100) for t in times]

In [ ]:
plt.figure(figsize=(16, 12))
plt.plot(times, results_exact, label="True", linestyle="--")
plt.plot(times, results_1_step, label="1 step")
plt.plot(times, results_5_step, label="5 step")
plt.plot(times, results_10_step, label="10 step")
plt.plot(times, results_20_step, label="20 step")
plt.plot(times, results_100_step, label="100 step")
plt.legend(fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Evolution time", fontsize=14)
plt.ylabel("<Y>", fontsize=14)
plt.show()